In [3]:
#--------------------Graphs/Tables/etc for Results Section--------------------#

print('Running Results Section')
import numpy as np
import pandas as pd
import csv
from scipy.stats import wilcoxon
import itertools as it
from math import comb

#--------------------Test for order effects--------------------#

print('Testing for order effects...')
cln_data_path = '../output/Data Background/'
both_orders_1s = pd.read_excel(cln_data_path + 'df_1s.xlsx')
both_orders_2s = pd.read_excel(cln_data_path + 'df_2s.xlsx')
both_orders_1s = both_orders_1s[both_orders_1s['is_seller'] == 1]
both_orders_2s = both_orders_2s[both_orders_2s['is_seller'] == 1]
both_orders_1s = both_orders_1s.groupby('session')[['pshh', 'pshl', 'order']].mean().reset_index()
both_orders_2s = both_orders_2s.groupby('session')[['pshh', 'pshl', 'order']].mean().reset_index()
both_orders_1s['pr_ph_1s'] = 0.5 * (both_orders_1s['pshh'] + both_orders_1s['pshl'])
both_orders_2s['pr_ph_2s'] = 0.5 * (both_orders_2s['pshh'] + both_orders_2s['pshl'])
df = pd.concat([both_orders_1s[['order', 'pr_ph_1s']], both_orders_2s[['pr_ph_2s']]], axis=1)
first = (df['order'] * df['pr_ph_1s'] + (1 - df['order']) * df['pr_ph_2s']).values
second = (df['order'] * df['pr_ph_2s'] + (1 - df['order']) * df['pr_ph_1s']).values
stat, p_value = wilcoxon(first, second)

with open("../output/Results Section/wilcoxon_test_vars.tex", "w") as f:
    f.write(f"\\newcommand{{\\wilcoxonstat}}{{{stat:.4f}}}\n")
    f.write(f"\\newcommand{{\\wilcoxonpvalue}}{{{p_value:.4f}}}\n")
print('Created wilcoxon_test_vars.tex')

#--------------------By-seller data on seller strategy space--------------------#

cln_data_path = '../output/Data Background/'
bysubj_sel_1s = pd.read_excel(cln_data_path + 'bysubj_sel_1s_first.xlsx')
bysubj_sel_2s = pd.read_excel(cln_data_path + 'bysubj_sel_2s_first.xlsx')
bysubj_sel_counts_1s = bysubj_sel_1s.groupby(['pshh', 'pshl']).size().reset_index(name='count')
bysubj_sel_counts_2s = bysubj_sel_2s.groupby(['pshh', 'pshl']).size().reset_index(name='count')

eqa = np.load('../output/Theory Section/eqa.npz')
loc_n1_lmix_eq = (eqa['lmix_pi_1s'][6], eqa['lmix_pi_1s'][4])
loc_n1_pool_eq = (eqa['pool_pi_1s'][6], eqa['pool_pi_1s'][4])
loc_n2_lmix_eq = (eqa['lmix_pi_2s'][9], eqa['lmix_pi_2s'][7])
loc_n2_hmix_eq = (eqa['hmix_pi_2s'][9], eqa['hmix_pi_2s'][7])
loc_n2_pool_eq = (eqa['pool_pi_2s'][9], eqa['pool_pi_2s'][7])

tex_code = r"""
\begin{figure}[htbp]\flushleft
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[scale=5]
\draw[thick,->] (-0.05,0) -- (1.1,0);
\node[below] at (0.5, 0) {$\mathbb{P}(p_H|v_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.1);
\node[above, rotate=90] at (0,0.5) {$\mathbb{P}(p_H|v_H)$};
\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);
\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at <<loc_n1_lmix_eq>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=red, fill=red ] at <<loc_n1_pool_eq>> {};
"""

for i in bysubj_sel_counts_1s.index:
    tex_code += r"""\node[draw=black, fill=black, circle, minimum size=""" + str(2*bysubj_sel_counts_1s['count'].loc[i]) + r"""pt, inner sep=0pt] at (""" + str(bysubj_sel_counts_1s['pshl'].loc[i]) + ", " + str(bysubj_sel_counts_1s['pshh'].loc[i]) + r""") {};
"""

tex_code += r"""            
\end{tikzpicture}
\caption{$n = 1$}
\end{subfigure}
\hspace{0.01\textwidth}
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[scale=5]
\draw[thick,->] (-0.05,0) -- (1.1,0);
\node[below] at (0.5, 0) {$\mathbb{P}(p_H|v_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.1);
\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);
\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at <<loc_n2_lmix_eq>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at <<loc_n2_hmix_eq>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=red, fill=red ] at <<loc_n2_pool_eq>> {};
"""

for i in bysubj_sel_counts_2s.index:
    tex_code += r"""\node[draw=black, fill=black, circle, minimum size=""" + str(2*bysubj_sel_counts_2s['count'].loc[i]) + r"""pt, inner sep=0pt] at (""" + str(bysubj_sel_counts_2s['pshl'].loc[i]) + ", " + str(bysubj_sel_counts_2s['pshh'].loc[i]) + r""") {};
"""

tex_code += r"""
\node[draw=black, fill=black, circle, minimum size=4pt, inner sep=0pt] at (1.4, 0.7) {};
\node[right] at (1.43, 0.7) {Data};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at (1.4, 0.5) {};
\node[right] at (1.43, 0.5) {\shortstack{Informative\\Nash}};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=red, fill=red ] at (1.4, 0.3) {};
\node[right] at (1.43, 0.3) {\shortstack{Uninformative\\Nash}};
\end{tikzpicture}
\caption{$n = 2$}
\end{subfigure}
\caption{Empirical Seller Strategies}
\label{sellerstrat_byseller_first}
    
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} Notice this notable note denotes noteworthy notices.
%\end{minipage}
    
\end{figure}
"""

replacements = {"<<loc_n1_lmix_eq>>": str(loc_n1_lmix_eq), 
                "<<loc_n2_lmix_eq>>": str(loc_n2_lmix_eq), 
                "<<loc_n2_hmix_eq>>": str(loc_n2_hmix_eq), 
                "<<loc_n1_pool_eq>>": str(loc_n1_pool_eq),
                "<<loc_n2_pool_eq>>": str(loc_n2_pool_eq)}

for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)

with open("../output/Results Section/sellerstrat_byseller_first.tex", "w") as file:
    file.write(tex_code)
print('Created sellerstrat_byseller_first.tex')

#--------------------By-buyer data on buyer strategy space--------------------#

cln_data_path = '../output/Data Background/'
bysubj_buy_1s = pd.read_excel(cln_data_path + 'bysubj_buy_1s_first.xlsx')
bysubj_buy_2s = pd.read_excel(cln_data_path + 'bysubj_buy_2s_first.xlsx')
bysubj_buy_counts_1s = bysubj_buy_1s.groupby(['pbhh', 'pbll']).size().reset_index(name='count')
bysubj_buy_counts_2s = bysubj_buy_2s.groupby(['pbhh', 'pbll', 'pbhb', 'pblb']).size().reset_index(name='count')

loc_n1_lmix_eq  = (eqa['lmix_pi_1s'][2], eqa['lmix_pi_1s'][0])
loc_n2_lmix_eq  = (eqa['lmix_pi_2s'][2], eqa['lmix_pi_2s'][0])
loc_n2_hmix_eq  = (eqa['hmix_pi_2s'][2], eqa['hmix_pi_2s'][0])
loc_n2_lmix_eq_ = (eqa['lmix_pi_2s'][5], eqa['lmix_pi_2s'][4])
loc_n2_hmix_eq_ = (eqa['hmix_pi_2s'][5], eqa['hmix_pi_2s'][4])
loc_n2_pool_eq_ = (eqa['pool_pi_2s'][5], eqa['pool_pi_2s'][4])

tex_code = r"""
\begin{figure}[htbp]\flushleft
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[scale=5]
\draw[thick,->] (-0.05,0) -- (1.1,0);
\node[below] at (0.5, 0) {$\mathbb{P}(\text{buy}|p_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.1);
\node[above, rotate=90] at (0,0.5) {$\mathbb{P}(\text{buy}|p_H)$};
\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);
\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\draw[line width=5pt, red] (1,0) -- (1,1/3);
\node[star,star points=5,star point ratio=2.5,scale=0.5,draw=blue,fill=blue] at <<loc_n1_lmix_eq>> {};
"""

for i in bysubj_buy_counts_1s.index:
    tex_code += r"""\node[draw=black, fill=black, circle, minimum size=""" + str(2*bysubj_buy_counts_1s['count'].loc[i]) + r"""pt, inner sep=0pt] at (""" + str(bysubj_buy_counts_1s['pbll'].loc[i]) + ", " + str(bysubj_buy_counts_1s['pbhh'].loc[i]) + r""") {};
"""

tex_code += r"""            
\end{tikzpicture}
%\caption{$n = 1$}
\end{subfigure}
\hspace{0.01\textwidth}
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[scale=5]
\draw[thick,->] (-0.05,0) -- (1.1,0);
\node[below] at (0.5, 0) {$\mathbb{P}(\text{buy}|p_L, p_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.1);
\node[above, rotate=90] at (0,0.5) {$\mathbb{P}(\text{buy}|p_H, p_H)$};
\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);
\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\draw[line width=5pt, red] (1,0) -- (1,1);
\node[star,star points=5,star point ratio=2.5,scale=0.5,draw=blue,fill=blue] at <<loc_n2_lmix_eq>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.5,draw=blue,fill=blue] at <<loc_n2_hmix_eq>> {};
"""

for i in bysubj_buy_counts_2s.index:
    tex_code += r"""\node[draw=black, fill=black, circle, minimum size=""" + str(2*bysubj_buy_counts_2s['count'].loc[i]) + r"""pt, inner sep=0pt] at (""" + str(bysubj_buy_counts_2s['pbll'].loc[i]) + ", " + str(bysubj_buy_counts_2s['pbhh'].loc[i]) + r""") {};
"""

tex_code += r"""
\node[draw=black, fill=black, circle, minimum size=4pt, inner sep=0pt] at (1.4, 0.7) {};
\node[right] at (1.43, 0.7) {Data};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at (1.4, 0.5) {};
\node[right] at (1.43, 0.5) {\shortstack{Informative\\Nash}};
\draw[line width=5pt, red] (1.35,0.3) -- (1.43,0.3);
\node[right] at (1.43, 0.3) {\shortstack{Uninformative\\Nash}};
\end{tikzpicture}
%\caption{$n = 2$}
\end{subfigure}

%\vskip\baselineskip

\begin{subfigure}[b]{0.4\textwidth}
\caption{$n = 1$}
\end{subfigure}
\hspace{0.01\textwidth}
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[scale=5]
\draw[thick,->] (-0.05,0) -- (1.1,0);
\node[below] at (0.5, 0) {$\mathbb{P}(\text{buy $p_L$}|p_H, p_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.1);
\node[above, rotate=90] at (0,0.5) {$\mathbb{P}(\text{buy $p_H$}|p_H, p_L)$};
\draw[thick] (-0.05,1)--(0,1);
\draw[thick] (1,0)--(1,-0.05);
\draw[thick] (1,0) -- (0,1);
\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\node[star,star points=5,star point ratio=2.5,scale=0.5,draw=blue,fill=blue] at <<loc_n2_lmix_eq_>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.5,draw=blue,fill=blue] at <<loc_n2_hmix_eq_>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.5,draw=red, fill=red ] at <<loc_n2_pool_eq_>> {};
"""

for i in bysubj_buy_counts_2s.index:
    tex_code += r"""\node[draw=black, fill=black, circle, minimum size=""" + str(2*bysubj_buy_counts_2s['count'].loc[i]) + r"""pt, inner sep=0pt] at (""" + str(bysubj_buy_counts_2s['pblb'].loc[i]) + ", " + str(bysubj_buy_counts_2s['pbhb'].loc[i]) + r""") {};
"""

tex_code += r"""
\node[draw=black, fill=black, circle, minimum size=4pt, inner sep=0pt] at (1.4, 0.7) {};
\node[right] at (1.43, 0.7) {Data};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at (1.4, 0.5) {};
\node[right] at (1.43, 0.5) {\shortstack{Informative\\Nash}};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=red,fill=red] at (1.4, 0.3) {};
\node[right] at (1.43, 0.3) {\shortstack{Uninformative\\Nash}};
\end{tikzpicture}
\caption{$n = 2$}
\end{subfigure}

\caption{Empirical Buyer Strategies}
\label{buyerstrat_bybuyer_first}
    
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} Notice this notable note denotes noteworthy notices.
%\end{minipage}
    
\end{figure}
"""

replacements = {"<<loc_n1_lmix_eq>>":  str(loc_n1_lmix_eq), 
                "<<loc_n2_lmix_eq>>":  str(loc_n2_lmix_eq), 
                "<<loc_n2_hmix_eq>>":  str(loc_n2_hmix_eq),
                "<<loc_n2_lmix_eq_>>": str(loc_n2_lmix_eq_),
                "<<loc_n2_hmix_eq_>>": str(loc_n2_hmix_eq_),
                "<<loc_n2_pool_eq_>>": str(loc_n2_pool_eq_)}

for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)

with open("../output/Results Section/buyerstrat_bybuyer_first.tex", "w") as file:
    file.write(tex_code)
print('Created buyerstrat_bybuyer_first.tex')

#--------------------Session data on seller strategy space WITH BOOTSTRAPPED CIs--------------------#

cln_data_path = '../output/Data Background/'
df_1s = pd.read_excel(cln_data_path + 'df_1s_first.xlsx')
df_2s = pd.read_excel(cln_data_path + 'df_2s_first.xlsx')
df_1s = df_1s[df_1s['is_seller'] == 1]
df_2s = df_2s[df_2s['is_seller'] == 1]
df_1s = df_1s[['session', 'pshh', 'pshl']]
df_2s = df_2s[['session', 'pshh', 'pshl']]

from sklearn.utils import resample
num_bootstrap_samples = 10000

sessions = []
actual_means_pshh = []
actual_means_pshl = []
quantile_5_pshh = []
quantile_95_pshh = []
quantile_5_pshl = []
quantile_95_pshl = []

for session, group in df_1s.groupby('session'):
    sessions.append(session)
    actual_means_pshh.append(group['pshh'].mean())
    actual_means_pshl.append(group['pshl'].mean())
    
    boot_means_pshh = [resample(group['pshh'], replace=True).mean() for _ in range(num_bootstrap_samples)]
    boot_means_pshl = [resample(group['pshl'], replace=True).mean() for _ in range(num_bootstrap_samples)]
    
    quantile_5_pshh.append(np.percentile(boot_means_pshh, 5))
    quantile_95_pshh.append(np.percentile(boot_means_pshh, 95))
    quantile_5_pshl.append(np.percentile(boot_means_pshl, 5))
    quantile_95_pshl.append(np.percentile(boot_means_pshl, 95))

result_1s = pd.DataFrame({
    'session': sessions,
    'pshh_avg': actual_means_pshh,
    'pshl_avg': actual_means_pshl,
    'pshh_q05': quantile_5_pshh,
    'pshh_q95': quantile_95_pshh,
    'pshl_q05': quantile_5_pshl,
    'pshl_q95': quantile_95_pshl
})

sessions = []
actual_means_pshh = []
actual_means_pshl = []
quantile_5_pshh = []
quantile_95_pshh = []
quantile_5_pshl = []
quantile_95_pshl = []

for session, group in df_2s.groupby('session'):
    sessions.append(session)
    actual_means_pshh.append(group['pshh'].mean())
    actual_means_pshl.append(group['pshl'].mean())
    
    boot_means_pshh = [resample(group['pshh'], replace=True).mean() for _ in range(num_bootstrap_samples)]
    boot_means_pshl = [resample(group['pshl'], replace=True).mean() for _ in range(num_bootstrap_samples)]
    
    quantile_5_pshh.append(np.percentile(boot_means_pshh, 5))
    quantile_95_pshh.append(np.percentile(boot_means_pshh, 95))
    quantile_5_pshl.append(np.percentile(boot_means_pshl, 5))
    quantile_95_pshl.append(np.percentile(boot_means_pshl, 95))

result_2s = pd.DataFrame({
    'session': sessions,
    'pshh_avg': actual_means_pshh,
    'pshl_avg': actual_means_pshl,
    'pshh_q05': quantile_5_pshh,
    'pshh_q95': quantile_95_pshh,
    'pshl_q05': quantile_5_pshl,
    'pshl_q95': quantile_95_pshl
})

# Nash choice probabilities
loc_n1_lmix_eq = (eqa['lmix_pi_1s'][6], eqa['lmix_pi_1s'][4])
loc_n1_pool_eq = (eqa['pool_pi_1s'][6], eqa['pool_pi_1s'][4])
loc_n2_lmix_eq = (eqa['lmix_pi_2s'][9], eqa['lmix_pi_2s'][7])
loc_n2_hmix_eq = (eqa['hmix_pi_2s'][9], eqa['hmix_pi_2s'][7])
loc_n2_pool_eq = (eqa['pool_pi_2s'][9], eqa['pool_pi_2s'][7])

tex_code = r"""
\begin{figure}[htbp]\flushleft
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[scale=5]
\draw[thick,->] (-0.05,0) -- (1.1,0);
\node[below] at (0.5, 0) {$\mathbb{P}(p_H|v_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.1);
\node[above, rotate=90] at (0,0.5) {$\mathbb{P}(p_H|v_H)$};
\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);
\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at <<loc_n1_lmix_eq>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=red, fill=red ] at <<loc_n1_pool_eq>> {};
"""

for i in result_1s.index:
    tex_code += r"""\node[draw=black, fill=black, circle, minimum size=4pt, inner sep=0pt] at (""" + str(result_1s['pshl_avg'].loc[i]) + ", " + str(result_1s['pshh_avg'].loc[i]) + r""") {};
\draw[thick, black] (""" + str(result_1s['pshl_avg'].loc[i]) + ", " + str(result_1s['pshh_q05'].loc[i]) + """) -- (""" + str(result_1s['pshl_avg'].loc[i]) + ", " + str(result_1s['pshh_q95'].loc[i]) + """);
\draw[thick, black] (""" + str(result_1s['pshl_q05'].loc[i]) + ", " + str(result_1s['pshh_avg'].loc[i]) + """) -- (""" + str(result_1s['pshl_q95'].loc[i]) + ", " + str(result_1s['pshh_avg'].loc[i]) + """);
"""

tex_code += r"""            
\end{tikzpicture}
\caption{$n = 1$}
\end{subfigure}
\hspace{0.01\textwidth}
\begin{subfigure}[b]{0.4\textwidth}
\begin{tikzpicture}[scale=5]
\draw[thick,->] (-0.05,0) -- (1.1,0);
\node[below] at (0.5, 0) {$\mathbb{P}(p_H|v_L)$};
\draw[thick,->] (0,-0.05) -- (0,1.1);
\draw[thick] (-0.05,1)--(1,1);
\draw[thick] (1,1)--(1,-0.05);
\node[left] at (-0.05,0) {0};
\node[left] at (-0.05,1) {1};
\node[below] at (0,-0.05) {0};
\node[below] at (1,-0.05) {1};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at <<loc_n2_lmix_eq>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at <<loc_n2_hmix_eq>> {};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=red, fill=red ] at <<loc_n2_pool_eq>> {};
"""

for i in result_2s.index:
    tex_code += r"""\node[draw=black, fill=black, circle, minimum size=4pt, inner sep=0pt] at (""" + str(result_2s['pshl_avg'].loc[i]) + ", " + str(result_2s['pshh_avg'].loc[i]) + r""") {};
\draw[thick, black] (""" + str(result_2s['pshl_avg'].loc[i]) + ", " + str(result_2s['pshh_q05'].loc[i]) + """) -- (""" + str(result_2s['pshl_avg'].loc[i]) + ", " + str(result_2s['pshh_q95'].loc[i]) + """);
\draw[thick, black] (""" + str(result_2s['pshl_q05'].loc[i]) + ", " + str(result_2s['pshh_avg'].loc[i]) + """) -- (""" + str(result_2s['pshl_q95'].loc[i]) + ", " + str(result_2s['pshh_avg'].loc[i]) + """);
"""

tex_code += r"""
\node[draw=black, fill=black, circle, minimum size=4pt, inner sep=0pt] at (1.4, 0.7) {};
\node[right] at (1.43, 0.7) {Data};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=blue,fill=blue] at (1.4, 0.5) {};
\node[right] at (1.43, 0.5) {\shortstack{Informative\\Nash}};
\node[star,star points=5,star point ratio=2.5,scale=0.4,draw=red,fill=red] at (1.4, 0.3) {};
\node[right] at (1.43, 0.3) {\shortstack{Uninformative\\Nash}};
\end{tikzpicture}
\caption{$n = 2$}
\end{subfigure}
\caption{Empirical Seller Strategies by Session}
\label{sellerstrat_bysession_CIs_first}
    
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} Notice this notable note denotes noteworthy notices.
%\end{minipage}
    
\end{figure}
"""

replacements = {"<<loc_n1_lmix_eq>>": str(loc_n1_lmix_eq), 
                "<<loc_n2_lmix_eq>>": str(loc_n2_lmix_eq), 
                "<<loc_n2_hmix_eq>>": str(loc_n2_hmix_eq), 
                "<<loc_n1_pool_eq>>": str(loc_n1_pool_eq),
                "<<loc_n2_pool_eq>>": str(loc_n2_pool_eq)}

for placeholder, value in replacements.items():
    tex_code = tex_code.replace(placeholder, value)

with open("../output/Results Section/sellerstrat_bysession_CIs_first.tex", "w") as file:
    file.write(tex_code)
print('Created sellerstrat_bysession_CIs_first.tex')

#--------------------Session data on seller strategy space without CIs--------------------#

# Just .data file for the QRE MLE graph
cln_data_path = '../output/Data Background/'
bysess_sel_1s_first = pd.read_excel(cln_data_path + 'bysess_sel_1s_first.xlsx')
bysess_sel_2s_first = pd.read_excel(cln_data_path + 'bysess_sel_2s_first.xlsx')

output_file = '../output/Results Section/sellerstrat_bysession_first.data'
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['pshl_1s', 'pshh_1s', 'pshl_2s', 'pshh_2s'])
    for row in zip(bysess_sel_1s_first['pshl'],
                   bysess_sel_1s_first['pshh'],
                   bysess_sel_2s_first['pshl'],
                   bysess_sel_2s_first['pshh']):
        writer.writerow(row)
print(f'Data written to {output_file}')

#--------------------Permutation Tests (Differences in Sample Means)--------------------#

cln_data_path = '../output/Data Background/'
bysess_sel_1s_first = pd.read_excel(cln_data_path + 'bysess_sel_1s_first.xlsx')
bysess_sel_2s_first = pd.read_excel(cln_data_path + 'bysess_sel_2s_first.xlsx')
dfs = [bysess_sel_1s_first, bysess_sel_2s_first]

pshh_sample = pd.concat([dfs[i]['pshh'].reset_index(drop=True)  for i in [0,1]], axis=1).values
pshl_sample = pd.concat([dfs[i]['pshl'].reset_index(drop=True)  for i in [0,1]], axis=1).values
x_sample    = pd.concat([dfs[i]['pr_ph'].reset_index(drop=True) for i in [0,1]], axis=1).values
inf_sample  = pd.concat([dfs[i]['inf'].reset_index(drop=True)   for i in [0,1]], axis=1).values

# function that does the permutation test
def test(sample, twosided=False):
    
    n = len(sample)
    num_permutations = comb(2 * n, n)
    
    idx = np.arange(2*n)
    idx = np.reshape(idx,(n,2))
    flat = list((idx.T).flatten())
    P = [list(x) for x in it.combinations(flat,n)]
    Q = [[y for y in flat if y not in x] for x in P]
    R = [[P[x],Q[x]] for x in range(len(P))]
    R = np.transpose(R,axes=(0,2,1))
    
    index1 = R//2
    index2 = R%2
    index = (index1,index2)
    
    permuted_sample = sample[index]
    
    # Create test stats
    teststats = (permuted_sample[:,:,0] - permuted_sample[:,:,1]).sum(axis=1)
    actual_teststat = teststats[0]
    
    # Count the number of permutations with equally/more extreme test stats
    if twosided == False:
        more_extreme = (teststats >= actual_teststat)
    elif twosided == True:
        more_extreme = (abs(teststats) >= abs(actual_teststat))
        
    num_more_extreme = more_extreme.sum()
    pvalue = num_more_extreme / num_permutations
    return [n, num_permutations, num_more_extreme, pvalue]


# Test results
pshh_testresult = test(pshh_sample)
pshl_testresult = test(pshl_sample)
x_testresult = test(x_sample)
inf_testresult = test(inf_sample, twosided=True)

Results = np.array([np.array(pshh_testresult),
                    np.array(pshl_testresult),
                    np.array(x_testresult),
                    np.array(inf_testresult)])

# Make Latex Table
row_labels = [r"""\shortstack{High-quality\\price decreases}""", 
              r"""\shortstack{Low-quality\\price decreases}""", 
              r"""\shortstack{Average price\\decreases}""",
              r"""\shortstack{Informativeness\\decreases}"""]

tex_code = r"""
\begin{figure}[htbp]
\renewcommand{\arraystretch}{2.5}
\begin{tabular}{lcccc}
\toprule
Outcome & Sample size & \shortstack{Number of\\permutations} & \shortstack{More extreme\\permutations} & p-value \\
\midrule 
"""

for label, row in zip(row_labels, Results):
    tex_code += f"{label} & {row[0]:.0f} & {row[1]:.0f} & {row[2]:.0f} & {row[3]:.4f} \\\\\n"
    
tex_code += r"""\bottomrule
\end{tabular}
\caption{Permutation Tests}
\label{permutation_tests_first}
    
%\begin{minipage}{0.8\textwidth}
%\footnotesize
%\textit{Notes:} Notice this notable note denotes noteworthy notices.
%\end{minipage}
    
\end{figure}
"""

with open("../output/Results Section/permutation_tests_first.tex", "w") as file:
    file.write(tex_code)
print('Created permutation_tests_first.tex')
print('\n')


Running Results Section
Testing for order effects...
Created wilcoxon_test_vars.tex
Created sellerstrat_byseller_first.tex
Created buyerstrat_bybuyer_first.tex


KeyboardInterrupt: 

In [ ]:
# #######################-----CURRENTLY UNUSED-----######################

# #--------------------Choice probabilities by round--------------------#

# cln_data_path = 'C://Users/Daniel/Box Sync/Desktop/Competition and Price Informativeness/Current/Experiment/data/clean/'
# byround_seller_1s = pd.read_excel(cln_data_path + 'byround_seller_1s.xlsx')
# byround_seller_2s = pd.read_excel(cln_data_path + 'byround_seller_2s.xlsx')
# byround_buyer_1s  = pd.read_excel(cln_data_path + 'byround_buyer_1s.xlsx')
# byround_buyer_2s  = pd.read_excel(cln_data_path + 'byround_buyer_2s.xlsx')

# output_file = 'choiceprobs_byround_first.data'
# with open(output_file, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['Period', 
#                      'pshh_1s', 
#                      'pshl_1s', 
#                      'muH_1s',
#                      'muL_1s',
#                      'inf_1s',
#                      'pshh_2s', 
#                      'pshl_2s', 
#                      'muH_2s',
#                      'muL_2s',
#                      'inf_2s',
#                      'pbhh_1s',
#                      'pbhh_2s',
#                      'pbhb_2s'])
#     for row in zip(byround_seller_1s['Period'], 
#                    byround_seller_1s['pshh'], 
#                    byround_seller_1s['pshl' ], 
#                    byround_seller_1s['muH'],
#                    byround_seller_1s['muL'],
#                    byround_seller_1s['inf'],
#                    byround_seller_2s['pshh'], 
#                    byround_seller_2s['pshl' ],
#                    byround_seller_2s['muH'],
#                    byround_seller_2s['muL'],
#                    byround_seller_2s['inf'],
#                    byround_buyer_1s['pbhh'],
#                    byround_buyer_2s['pbhh'],
#                    byround_buyer_2s['pbhb']):
#         writer.writerow(row)
# print(f'Data written to {output_file}')

# tex_code = r"""
# \begin{figure}[htbp]\flushleft

# \begin{subfigure}[b]{0.4\textwidth}
# \begin{tikzpicture}[scale=0.8]
# \begin{axis}[no markers, ylabel={Seller Probabilities}, xtick={1,2,3,4,5,6,7,8,9,10}, xticklabels={}, ytick={0,1}, yticklabels={0,1}, ultra thick, xmin=0, xmax=11, ymin=-0.1, ymax=1.1]
# \addplot [color=blue, solid] table [x=Period, y=pshh_1s, col sep=comma, unbounded coords=jump] {graphs/choiceprobs_byround_first.data};
# \addplot [color=red, dashed] table [x=Period, y=pshl_1s, col sep=comma, unbounded coords=jump] {graphs/choiceprobs_byround_first.data};
# \end{axis}
# \end{tikzpicture}
# \end{subfigure}
# \hspace{0.01\textwidth}

# \begin{subfigure}[b]{0.4\textwidth}
# \begin{tikzpicture}[scale=0.8]
# \begin{axis}[no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align={left}, xtick={1,2,3,4,5,6,7,8,9,10}, xticklabels={}, ytick={0,1}, yticklabels={}, ultra thick, xmin=0, xmax=11, ymin=-0.1, ymax=1.1]
# \addplot [color=blue, solid] table [x=Period, y=pshh_2s, col sep=comma, unbounded coords=jump] {graphs/choiceprobs_byround_first.data};
# \addlegendentry{$\mathbb{P}(p_H|v_H)$}
# \addplot [color=red, dashed] table [x=Period, y=pshl_2s, col sep=comma, unbounded coords=jump] {graphs/choiceprobs_byround_first.data};
# \addlegendentry{$\mathbb{P}(p_H|v_L)$}
# \end{axis}
# \end{tikzpicture}
# \end{subfigure}

# \begin{subfigure}[b]{0.4\textwidth}
# \begin{tikzpicture}[scale=0.8]
# \begin{axis}[no markers, xlabel={Round}, ylabel={Buyer Probabilities}, xtick={1,2,3,4,5,6,7,8,9,10}, xticklabels={1,2,3,4,5,6,7,8,9,10}, ytick={0,1}, yticklabels={0,1}, ultra thick, xmin=0, xmax=11, ymin=-0.1, ymax=1.1]
# \addplot [color=green, solid] table [x=Period, y=pbhh_1s, col sep=comma, unbounded coords=jump] {graphs/choiceprobs_byround_first.data};
# \end{axis}
# \end{tikzpicture}
# \caption{One Seller}
# \end{subfigure}
# \hspace{0.01\textwidth}

# \begin{subfigure}[b]{0.4\textwidth}
# \begin{tikzpicture}[scale=0.8]
# \begin{axis}[no markers, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, xlabel={Round}, legend cell align={left}, xtick={1,2,3,4,5,6,7,8,9,10}, xticklabels={1,2,3,4,5,6,7,8,9,10}, ytick={0,1}, yticklabels={}, ultra thick, xmin=0, xmax=11, ymin=-0.1, ymax=1.1]
# \addplot [color=green, solid] table [x=Period, y=pbhh_2s, col sep=comma, unbounded coords=jump] {graphs/choiceprobs_byround_first.data};
# \addlegendentry{$\mathbb{P}(\text{buy} \ p_H|p_H)$}
# \addplot [color=orange, dashed] table [x=Period, y=pbhb_2s, col sep=comma, unbounded coords=jump] {graphs/choiceprobs_byround_first.data};
# \addlegendentry{$\mathbb{P}(\text{buy} \ p_H|p_H, p_L)$}
# \end{axis}
# \end{tikzpicture}
# \caption{Two Sellers}
# \end{subfigure}

# \caption{Choice Probabilites by Round}
# \label{choiceprobs_byround_first}
# %\begin{minipage}{0.8\textwidth}
# %\footnotesize
# %\textit{Notes:} Notice this notable note denotes noteworthy notices.
# %\end{minipage}
# \end{figure}
# """

# with open("choiceprobs_byround_first.tex", "w") as file:
#     file.write(tex_code)

# #--------------------Session data on seller strategy space--------------------#

# cln_data_path = 'C://Users/Daniel/Box Sync/Desktop/Competition and Price Informativeness/Current/Experiment/data/clean/'
# bysession_seller_1s = pd.read_excel(cln_data_path + 'bysession_seller_1s.xlsx')
# bysession_seller_2s = pd.read_excel(cln_data_path + 'bysession_seller_2s.xlsx')

# # Nash choice probabilities
# loc_n1_lowmix_eq = (lowmix_phl[lowmix_n == 1][0], lowmix_phh[lowmix_n == 1][0])
# loc_n1_pool_eq = (pool_phl[pool_n == 1][0], pool_phh[pool_n == 1][0])

# loc_n2_lowmix_eq = (lowmix_phl[lowmix_n == 2][0], lowmix_phh[lowmix_n == 2][0])
# loc_n2_highmix_eq = (highmix_phl[highmix_n == 2][0], highmix_phh[highmix_n == 2][0])
# loc_n2_pool_eq = (pool_phl[pool_n == 2][0], pool_phh[pool_n == 2][0])

# # Session data to csv
# output_file = 'sellerstrat_bysession_first.data'
# with open(output_file, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['pshh_1s', 
#                      'pshl_1s', 
#                      'pshh_2s', 
#                      'pshl_2s'])
#     for row in zip(bysession_seller_1s['pshh'], 
#                    bysession_seller_1s['pshl'], 
#                    bysession_seller_2s['pshh'], 
#                    bysession_seller_2s['pshl']):
#         writer.writerow(row)
# print(f'Data written to {output_file}')

# tex_code = r"""
# \begin{figure}[htbp]\flushleft
# \begin{subfigure}[b]{0.4\textwidth}
# \begin{tikzpicture}[scale=0.8]
# \begin{axis}[xlabel={$\mathbb{P}(p_H|v_L)$}, ylabel={$\mathbb{P}(p_H|v_H)$}, only marks, xtick={0,1}, xticklabels={0,1}, ytick={0,1}, yticklabels={0,1}, ultra thick, xmin=-0.05, xmax=1.05, ymin=-0.05, ymax=1.05]
# \addplot[mark=*, mark options={color=black, fill=black}] table[x=pshl_1s, y=pshh_1s, col sep=comma] {graphs/sellerstrat_bysession_first.data}; 
# \addplot[only marks, mark=star, mark options={color=blue, fill=blue}, mark size=5pt] coordinates {<<loc_n1_lowmix_eq>>};
# \addplot[only marks, mark=star, mark options={color=red, fill=red}, mark size=5pt] coordinates {<<loc_n1_pool_eq>>};
# \end{axis}
# \end{tikzpicture}
# \caption{$n = 1$}
# \end{subfigure}
# \hspace{0.01\textwidth}
# \begin{subfigure}[b]{0.4\textwidth}
# \begin{tikzpicture}[scale=0.8]
# \begin{axis}[xlabel={$\mathbb{P}(p_H|v_L)$}, only marks, xtick={0,1}, xticklabels={0,1}, ytick={0,1}, yticklabels={}, ultra thick, xmin=-0.05, xmax=1.05, ymin=-0.05, ymax=1.05, legend style={at={(1.1,0.5)}, anchor=west, legend columns=1}, legend cell align=left]
# \addplot[only marks, mark=*, mark options={color=black, fill=black}] table[x=pshl_2s, y=pshh_2s, col sep=comma] {graphs/sellerstrat_bysession_first.data}; 
# \addlegendentry{Data}
# \addplot[only marks, mark=star, mark options={color=red, fill=red}, mark size=5pt] coordinates {<<loc_n2_pool_eq>>};
# \addlegendentry{\shortstack{Uninformative\\Nash}}
# \addplot[only marks, mark=star, mark options={color=blue, fill=blue}, mark size=5pt] coordinates {<<loc_n2_lowmix_eq>>};
# \addlegendentry{\shortstack{Informative\\Nash}}
# \addplot[only marks, mark=star, mark options={color=blue, fill=blue}, mark size=5pt] coordinates {<<loc_n2_highmix_eq>>};
# \end{axis}
# \end{tikzpicture}
# \caption{$n = 2$}
# \end{subfigure}
# \caption{Equilibrium Selection}
# \label{sellerstrat_bysession_first}
    
# %\begin{minipage}{0.8\textwidth}
# %\footnotesize
# %\textit{Notes:} Notice this notable note denotes noteworthy notices.
# %\end{minipage}
    
# \end{figure}
# """

# replacements = {"<<loc_n1_lowmix_eq>>": str(loc_n1_lowmix_eq), 
#                 "<<loc_n2_lowmix_eq>>": str(loc_n2_lowmix_eq), 
#                 "<<loc_n2_highmix_eq>>": str(loc_n2_highmix_eq), 
#                 "<<loc_n1_pool_eq>>": str(loc_n1_pool_eq),
#                 "<<loc_n2_pool_eq>>": str(loc_n2_pool_eq)}

# for placeholder, value in replacements.items():
#     tex_code = tex_code.replace(placeholder, value)

# with open("sellerstrat_bysession_first.tex", "w") as file:
#     file.write(tex_code)